In [33]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.linear_model import LogisticRegression
import yfinance as yf
import hvplot.pandas
from sklearn.preprocessing import StandardScaler

In [107]:
# Load the CSV data into a DataFrame
file_path = Path("CorporateStockSplits_10yr_TechOnly.csv")
CSV_Split_df = pd.read_csv(file_path, infer_datetime_format = True )

# had to remove date as index because its not unique and the one hot encoder requires unique index values. 
# , index_col = "Date", infer_datetime_format = True 

# Display sample data
Full_Split_df = CSV_Split_df.dropna()
Full_Split_df

,Date,Symbol,Company Name,Type,Split Ratio,Split Number,Industry,Price on day of Split,Price Prior to Split,Date4Q,...,Price4Q,Date3Q,Volume3Q,Price3Q,Date2Q,Volume2Q,Price2Q,Date1Q,Volume1Q,Price1Q
0,4/18/2023,INTC,Intel Corporation,None,None,1,Technology,31.83,31.83,4/22/2022,...,46.54,7/21/2022,29661198,40.61,10/19/2022,33882274,26.00,1/18/2023,44199580,28.81
1,4/18/2023,MXL,MaxLinear Inc.,None,None,1,Technology,31.59,31.59,4/22/2022,...,44.14,7/21/2022,693403,40.41,10/19/2022,807336,30.45,1/18/2023,489581,36.77
2,4/18/2023,MRVL,Marvell Technology Group Ltd.,None,None,1,Technology,42.24,42.24,4/22/2022,...,58.44,7/21/2022,7900197,54.57,10/19/2022,8271283,38.10,1/18/2023,7445092,40.49
3,4/18/2023,STM,STMicroelectronics NV,None,None,1,Technology,50.59,50.59,4/22/2022,...,37.47,7/21/2022,3288557,35.22,10/19/2022,3486155,31.82,1/18/2023,4139197,42.29
4,4/18/2023,MU,Micron Technology Inc.,None,None,1,Technology,61.93,61.93,4/22/2022,...,69.41,7/21/2022,12924682,63.64,10/19/2022,14786102,53.03,1/18/2023,10035267,56.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,6/15/2022,SBIG,Springbig Holdings Inc,Forward,2 for 1,2,Technology,4.50,9.00,6/15/2021,...,9.70,9/15/2021,934,9.70,12/17/2021,16316,9.86,3/17/2022,27561,19.86
59,6/23/2022,FTNT,Fortinet Inc,Forward,5 for 1,5,Technology,56.80,284.00,6/23/2021,...,240.05,9/23/2021,977481,310.55,12/27/2021,1019028,367.65,3/25/2022,858572,332.75
60,6/29/2022,SHOP,Shopify Inc,Forward,10 for 1,10,Technology,33.05,330.50,6/29/2021,...,1483.00,9/29/2021,1005928,1346.60,12/31/2021,471261,1377.40,3/31/2022,2222722,676.00
61,9/14/2022,PANW,Palo Alto Networks Inc,Forward,3 for 1,3,Technology,182.06,546.18,9/14/2021,...,484.83,12/15/2021,1569519,540.30,3/18/2022,2167555,577.02,6/16/2022,1367100,466.32


In [108]:
# Have to remove reverse split companies because that is going to throw off our test population.
Split_df = Full_Split_df.loc[Full_Split_df["Type"].isin(['None', 'Forward'])] 

Split_df

,Date,Symbol,Company Name,Type,Split Ratio,Split Number,Industry,Price on day of Split,Price Prior to Split,Date4Q,...,Price4Q,Date3Q,Volume3Q,Price3Q,Date2Q,Volume2Q,Price2Q,Date1Q,Volume1Q,Price1Q
0,4/18/2023,INTC,Intel Corporation,None,None,1,Technology,31.83,31.83,4/22/2022,...,46.54,7/21/2022,29661198,40.61,10/19/2022,33882274,26.00,1/18/2023,44199580,28.81
1,4/18/2023,MXL,MaxLinear Inc.,None,None,1,Technology,31.59,31.59,4/22/2022,...,44.14,7/21/2022,693403,40.41,10/19/2022,807336,30.45,1/18/2023,489581,36.77
2,4/18/2023,MRVL,Marvell Technology Group Ltd.,None,None,1,Technology,42.24,42.24,4/22/2022,...,58.44,7/21/2022,7900197,54.57,10/19/2022,8271283,38.10,1/18/2023,7445092,40.49
3,4/18/2023,STM,STMicroelectronics NV,None,None,1,Technology,50.59,50.59,4/22/2022,...,37.47,7/21/2022,3288557,35.22,10/19/2022,3486155,31.82,1/18/2023,4139197,42.29
4,4/18/2023,MU,Micron Technology Inc.,None,None,1,Technology,61.93,61.93,4/22/2022,...,69.41,7/21/2022,12924682,63.64,10/19/2022,14786102,53.03,1/18/2023,10035267,56.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,6/15/2022,SBIG,Springbig Holdings Inc,Forward,2 for 1,2,Technology,4.50,9.00,6/15/2021,...,9.70,9/15/2021,934,9.70,12/17/2021,16316,9.86,3/17/2022,27561,19.86
59,6/23/2022,FTNT,Fortinet Inc,Forward,5 for 1,5,Technology,56.80,284.00,6/23/2021,...,240.05,9/23/2021,977481,310.55,12/27/2021,1019028,367.65,3/25/2022,858572,332.75
60,6/29/2022,SHOP,Shopify Inc,Forward,10 for 1,10,Technology,33.05,330.50,6/29/2021,...,1483.00,9/29/2021,1005928,1346.60,12/31/2021,471261,1377.40,3/31/2022,2222722,676.00
61,9/14/2022,PANW,Palo Alto Networks Inc,Forward,3 for 1,3,Technology,182.06,546.18,9/14/2021,...,484.83,12/15/2021,1569519,540.30,3/18/2022,2167555,577.02,6/16/2022,1367100,466.32


In [109]:
enc = OneHotEncoder(sparse=False)

In [110]:
categorical = ["Type"]

In [111]:
encoded_data = enc.fit_transform(Split_df[categorical])

In [112]:
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical)
)

# Display sample data
encoded_df.head()

C:\Users\danie\anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Type_Forward,Type_None
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [113]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
Full_df = pd.concat(
    [Split_df[["Price on day of Split", "Price Prior to Split", "Split Number","Volume4Q","Price4Q","Volume3Q","Price3Q","Volume2Q","Price2Q","Volume1Q","Price1Q"]], encoded_df],axis=1)
	# "Date4Q" "Date3Q" "Date2Q" "Date1Q"		

# Display sample data
Full_df.head()

,Price on day of Split,Price Prior to Split,Split Number,Volume4Q,Price4Q,Volume3Q,Price3Q,Volume2Q,Price2Q,Volume1Q,Price1Q,Type_Forward,Type_None
0,31.83,31.83,1,28219860,46.54,29661198,40.61,33882274,26.00,44199580,28.81,0.0,1.0
1,31.59,31.59,1,494735,44.14,693403,40.41,807336,30.45,489581,36.77,0.0,1.0
2,42.24,42.24,1,8638103,58.44,7900197,54.57,8271283,38.10,7445092,40.49,0.0,1.0
3,50.59,50.59,1,3460450,37.47,3288557,35.22,3486155,31.82,4139197,42.29,0.0,1.0
4,61.93,61.93,1,18682432,69.41,12924682,63.64,14786102,53.03,10035267,56.51,0.0,1.0


In [114]:
# Creating the features set X
X = Full_df.drop(columns=["Type_Forward","Type_None"], axis=1)

# Display sample data
X

,Price on day of Split,Price Prior to Split,Split Number,Volume4Q,Price4Q,Volume3Q,Price3Q,Volume2Q,Price2Q,Volume1Q,Price1Q
0,31.83,31.83,1,28219860,46.54,29661198,40.61,33882274,26.00,44199580,28.81
1,31.59,31.59,1,494735,44.14,693403,40.41,807336,30.45,489581,36.77
2,42.24,42.24,1,8638103,58.44,7900197,54.57,8271283,38.10,7445092,40.49
3,50.59,50.59,1,3460450,37.47,3288557,35.22,3486155,31.82,4139197,42.29
4,61.93,61.93,1,18682432,69.41,12924682,63.64,14786102,53.03,10035267,56.51
...,...,...,...,...,...,...,...,...,...,...,...
58,4.50,9.00,2,8145,9.70,934,9.70,16316,9.86,27561,19.86
59,56.80,284.00,5,781067,240.05,977481,310.55,1019028,367.65,858572,332.75
60,33.05,330.50,10,1058087,1483.00,1005928,1346.60,471261,1377.40,2222722,676.00
61,182.06,546.18,3,2464777,484.83,1569519,540.30,2167555,577.02,1367100,466.32


In [115]:
# Creating the target set y
y = encoded_df["Type_None"]

# Display sample data
y.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: Type_None, dtype: float64

In [116]:
# Split the preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [117]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [118]:
model = tree.DecisionTreeClassifier()

In [119]:
model = model.fit(X_train_scaled, y_train)

In [120]:
predictions = model.predict(X_test_scaled)

In [121]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        15
         1.0       1.00      1.00      1.00         1

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [122]:
# Trying out a logistic Regression Model

In [123]:
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [124]:
 LogisticRegression(random_state=1)

LogisticRegression(random_state=1)

In [125]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [126]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5319148936170213
Testing Data Score: 0.3125


In [127]:
predictions_LR = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions_LR, "Actual": y_test})

,Prediction,Actual
24,0.0,0.0
39,1.0,0.0
51,1.0,0.0
27,0.0,0.0
61,1.0,0.0
2,1.0,1.0
21,1.0,0.0
42,0.0,0.0
41,1.0,0.0
44,1.0,0.0


In [128]:
# Display the accuracy score for the Logistic Regression Model.
accuracy_score(y_test, predictions_LR)

0.3125

In [129]:
# As we are able to see with an accuracy score of 31.25% this model isnt the most accurate thing in the world.  
print("Your model is accurate ",accuracy_score(y_test, predictions_LR)*100 , "% of the time every time")

Your model is accurate  31.25 % of the time every time


In [130]:
# I am trying the KMeans Model as the logistic regression model is giving me a low accuracy. 

In [131]:
Split_KMeans = Full_df.drop(columns=["Type_Forward","Type_None"], axis=1)

In [132]:
model = KMeans(n_clusters =10, random_state=0)

In [133]:
Split_DataScaled = StandardScaler().fit_transform(
    Split_KMeans[["Price on day of Split","Price Prior to Split","Split Number",	"Volume4Q",	"Price4Q",	"Volume3Q",	"Price3Q",	"Volume2Q",	"Price2Q",	"Volume1Q"	,"Price1Q"]])

In [134]:
model.fit(Split_DataScaled)

KMeans(n_clusters=10, random_state=0)

In [135]:
SplitPredictions = model.predict(Split_DataScaled)

In [136]:
SplitPredictions 

array([9, 5, 8, 5, 8, 8, 1, 8, 5, 8, 8, 8, 8, 8, 1, 4, 2, 2, 2, 5, 5, 5,
       5, 0, 5, 5, 0, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 6, 0, 5, 0, 5, 5, 7, 4, 5, 4, 5, 5, 5, 5, 4, 3, 4, 5])

In [137]:
Full_df['clusters']= SplitPredictions

# Review the DataFrame
Full_df.head()

,Price on day of Split,Price Prior to Split,Split Number,Volume4Q,Price4Q,Volume3Q,Price3Q,Volume2Q,Price2Q,Volume1Q,Price1Q,Type_Forward,Type_None,clusters
0,31.83,31.83,1,28219860,46.54,29661198,40.61,33882274,26.00,44199580,28.81,0.0,1.0,9
1,31.59,31.59,1,494735,44.14,693403,40.41,807336,30.45,489581,36.77,0.0,1.0,5
2,42.24,42.24,1,8638103,58.44,7900197,54.57,8271283,38.10,7445092,40.49,0.0,1.0,8
3,50.59,50.59,1,3460450,37.47,3288557,35.22,3486155,31.82,4139197,42.29,0.0,1.0,5
4,61.93,61.93,1,18682432,69.41,12924682,63.64,14786102,53.03,10035267,56.51,0.0,1.0,8


In [138]:

KM_Full_df = pd.concat(
    [Split_df[["Symbol", "Company Name","Type"]], Full_df],axis=1)

KM_Full_df.head()

,Symbol,Company Name,Type,Price on day of Split,Price Prior to Split,Split Number,Volume4Q,Price4Q,Volume3Q,Price3Q,Volume2Q,Price2Q,Volume1Q,Price1Q,Type_Forward,Type_None,clusters
0,INTC,Intel Corporation,None,31.83,31.83,1,28219860,46.54,29661198,40.61,33882274,26.00,44199580,28.81,0.0,1.0,9
1,MXL,MaxLinear Inc.,None,31.59,31.59,1,494735,44.14,693403,40.41,807336,30.45,489581,36.77,0.0,1.0,5
2,MRVL,Marvell Technology Group Ltd.,None,42.24,42.24,1,8638103,58.44,7900197,54.57,8271283,38.10,7445092,40.49,0.0,1.0,8
3,STM,STMicroelectronics NV,None,50.59,50.59,1,3460450,37.47,3288557,35.22,3486155,31.82,4139197,42.29,0.0,1.0,5
4,MU,Micron Technology Inc.,None,61.93,61.93,1,18682432,69.41,12924682,63.64,14786102,53.03,10035267,56.51,0.0,1.0,8


In [139]:
KM_Full_df.hvplot.scatter(x="Type",y="clusters",by="Company Name",height = 500, width = 1500)

:NdOverlay   [Company Name]
   :Scatter   [Type]   (clusters)

In [156]:
KM_Split_Date = Full_Split_df.loc[Full_Split_df["Type"].isin(['Forward'])]

KM_Split_Date['FormatDate'] = pd.to_datetime(KM_Split_Date["Date"])
KM_Split_Date['SplitQuarter'] = KM_Split_Date['FormatDate'].dt.quarter


KM_Split_Date.head()


C:\Users\danie\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\danie\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Date,Symbol,Company Name,Type,Split Ratio,Split Number,Industry,Price on day of Split,Price Prior to Split,Date4Q,...,Volume3Q,Price3Q,Date2Q,Volume2Q,Price2Q,Date1Q,Volume1Q,Price1Q,FormatDate,SplitQuarter
19,5/1/2012,ASUR,Asure Software Inc,Forward,2 for 1,2,Technology,4.71,9.42,5/2/2011,...,2500,4.00,11/3/2011,9340,4.82,2/1/2012,3275,9.44,2012-05-01,2
20,2/25/2013,DDD,3d Systems Corp,Forward,2 for 1,2,Technology,34.55,69.10,2/26/2012,...,1176362,42.40,8/29/2012,1358195,59.06,11/27/2012,1981537,60.32,2013-02-25,1
21,3/21/2013,TRMB,Trimble Inc,Forward,2 for 1,2,Technology,29.44,58.88,3/21/2012,...,514967,47.14,9/24/2012,633223,49.02,12/21/2012,1429294,59.38,2013-03-21,1
22,4/15/2013,RDWR,Radware Ltd.,Forward,2 for 1,2,Technology,14.40,28.80,4/15/2012,...,67915,36.20,10/17/2012,173003,33.28,1/15/2013,209200,34.34,2013-04-15,2
23,4/18/2013,CRM,Salesforce Inc,Forward,4 for 1,4,Technology,41.03,164.12,4/18/2012,...,2519580,139.12,10/22/2012,1605520,147.64,1/18/2013,1586413,170.04,2013-04-18,2


In [164]:
KM_Split_Date.hvplot.scatter(x="SplitQuarter",y="Split Ratio",by="Company Name",rot = 90,height = 600, width = 1000,xticks=[1, 2, 3, 4])

:NdOverlay   [Company Name]
   :Scatter   [SplitQuarter]   (Split Ratio)